### Final Prediction model

- including normal version and class version

In [2]:
from keras.models import load_model
#from helpers import resize_to_fit
from imutils import paths
import numpy as np
import imutils
import cv2
import pickle

## model file
recognition_model = "captcha_model.hdf5"

### label file
Label_model = "model_labels.dat"

## put testing image here
Testing_image_file = "Testing_image"

## Load up the model label

with open(Label_model,"rb") as f:
    lb = pickle.load(f)
    
## Load the recognition model
model = load_model(recognition_model)

### read image from file
image = cv2.imread(Testing_image_file+"/image.jpg")
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

## Add extra padding to margin, color = white
image = cv2.copyMakeBorder(image,20,20,20,20,cv2.BORDER_CONSTANT, value=(255,255,255))

### threshold the image (convert to pure black and white)

thresh = cv2.threshold(image,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

### find the contours (continuous blobs of pixels) 
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

### Hack for compatibility with different OpenCV version
contours = contours[0] if imutils.is_cv2() else contours[1]

letter_image_regions = []
### Make sure not greater than 6 letters
letter_left = 6

for contour in contours:
    ### Get the bounding box
    (x,y,w,h) = cv2.boundingRect(contour)
    
    if w > 10:
        # Compare the width and height of the contour to detect letters that
        # are conjoined into one chunk
        if w >=125 and letter_left >=5:
            letters_width = int(w / 5)
            letter_image_regions.append((x, y, letters_width, h))
            letter_image_regions.append((x + letters_width, y, letters_width, h))
            letter_image_regions.append((x + letters_width+letters_width, y, letters_width, h))
            letter_image_regions.append((x + letters_width+letters_width+letters_width, y, letters_width, h))
            letter_image_regions.append((x + letters_width+letters_width+letters_width+letters_width, y, letters_width, h))

            letter_left = letter_left - 5

        elif w >=95 and letter_left >= 4:
            letters_width = int(w / 4)
            letter_image_regions.append((x, y, letters_width, h))
            letter_image_regions.append((x + letters_width, y, letters_width, h))
            letter_image_regions.append((x + letters_width+letters_width, y, letters_width, h))
            letter_image_regions.append((x + letters_width+letters_width+letters_width, y, letters_width, h))

            letter_left = letter_left - 4

        elif w >68 and letter_left >= 3:
            letters_width = int(w / 3)
            letter_image_regions.append((x, y, letters_width, h))
            letter_image_regions.append((x + letters_width, y, letters_width, h))
            letter_image_regions.append((x + letters_width+letters_width, y, letters_width, h))

            letter_left = letter_left - 3

        elif w >=38 and letter_left >= 2:
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))

            letter_left = letter_left - 2



        else:
            letter_image_regions.append((x, y, w, h))
            letter_left = letter_left - 1
           
    ### If the letters != 6, skip the image 
if len(letter_image_regions) == 6:
    #print(len(letter_image_regions))
    
    ## Sort the letter image based on x coordinate
    letter_image_regions = sorted(letter_image_regions,key = lambda x:x[0])
    
    ## Create an output image and a list to hold predicted letters
    
    output = cv2.merge([image]*3)
    
    predictions = []
    
    ## loop over the letters
    for letter_bounding_box in letter_image_regions:
        
        x,y,w,h = letter_bounding_box
        
        ## Extract the letter from original image with 2-pixel margin around the edge
        letter_image = image[y - 2:y + h + 2, x - 2:x + w + 2]
        
        ## Resize the letter to 20*20 for prediction
        
        letter_image = cv2.resize(letter_image,(20,20))
        
        ## Turn single image into a 4d array for keras model predict
        letter_image = np.reshape(letter_image, (1,20,20,1))
        
        ## Predict the letters
        prediction = model.predict(letter_image)
        
        ## Convert the one-hot-encoded prediction back to a normal letter
        ## return a (1,) ndarray , use [0] to get value
        letter = lb.inverse_transform(prediction)[0]
        predictions.append(letter)
        
        # print the captcha text
        
    captcha_text = "".join(predictions)
        
    print(captcha_text)
    
    
    
        
        
        
    
            
        
    

ELURNN


In [4]:
## show the image 
cv2.imshow("Output", image)
cv2.waitKey()


-1

## Class version

In [0]:
from keras.models import load_model
#from helpers import resize_to_fit
from imutils import paths
import numpy as np
import imutils
import cv2
import pickle
class Amazon_Captcha_Recognition:
    
    def __init__(self):
        ## model file
        self.recognition_model = "captcha_model.hdf5"
        
        ### label file
        self.Label_model = "model_labels.dat"
        
        self.model = load_model(self.recognition_model)
    




    def Captcha_Recognition(self,IMAGE_PATH):


        ## Load up the model label

        with open(self.Label_model,"rb") as f:
            lb = pickle.load(f)

        ## Load the recognition model
        ### model = load_model(self.recognition_model)
        model = self.model
        ### read image from file
        image = cv2.imread(IMAGE_PATH)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

        ## Add extra padding to margin, color = white
        image = cv2.copyMakeBorder(image,20,20,20,20,cv2.BORDER_CONSTANT, value=(255,255,255))

        ### threshold the image (convert to pure black and white)

        thresh = cv2.threshold(image,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

        ### find the contours (continuous blobs of pixels) 
        contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        ### Hack for compatibility with different OpenCV version
        contours = contours[0] if imutils.is_cv2() else contours[1]

        letter_image_regions = []
        ### Make sure not greater than 6 letters
        letter_left = 6

        for contour in contours:
            ### Get the bounding box
            (x,y,w,h) = cv2.boundingRect(contour)

            if w > 10:
                # Compare the width and height of the contour to detect letters that
                # are conjoined into one chunk
                if w >=125 and letter_left >=5:
                    letters_width = int(w / 5)
                    letter_image_regions.append((x, y, letters_width, h))
                    letter_image_regions.append((x + letters_width, y, letters_width, h))
                    letter_image_regions.append((x + letters_width+letters_width, y, letters_width, h))
                    letter_image_regions.append((x + letters_width+letters_width+letters_width, y, letters_width, h))
                    letter_image_regions.append((x + letters_width+letters_width+letters_width+letters_width, y, letters_width, h))

                    letter_left = letter_left - 5

                elif w >=95 and letter_left >= 4:
                    letters_width = int(w / 4)
                    letter_image_regions.append((x, y, letters_width, h))
                    letter_image_regions.append((x + letters_width, y, letters_width, h))
                    letter_image_regions.append((x + letters_width+letters_width, y, letters_width, h))
                    letter_image_regions.append((x + letters_width+letters_width+letters_width, y, letters_width, h))

                    letter_left = letter_left - 4

                elif w >68 and letter_left >= 3:
                    letters_width = int(w / 3)
                    letter_image_regions.append((x, y, letters_width, h))
                    letter_image_regions.append((x + letters_width, y, letters_width, h))
                    letter_image_regions.append((x + letters_width+letters_width, y, letters_width, h))

                    letter_left = letter_left - 3

                elif w >=38 and letter_left >= 2:
                    half_width = int(w / 2)
                    letter_image_regions.append((x, y, half_width, h))
                    letter_image_regions.append((x + half_width, y, half_width, h))

                    letter_left = letter_left - 2



                else:
                    letter_image_regions.append((x, y, w, h))
                    letter_left = letter_left - 1

            ### If the letters != 6, skip the image 
        if len(letter_image_regions) == 6:
            #print(len(letter_image_regions))

            ## Sort the letter image based on x coordinate
            letter_image_regions = sorted(letter_image_regions,key = lambda x:x[0])

            ## Create an output image and a list to hold predicted letters

            output = cv2.merge([image]*3)

            predictions = []

            ## loop over the letters
            for letter_bounding_box in letter_image_regions:

                x,y,w,h = letter_bounding_box

                ## Extract the letter from original image with 2-pixel margin around the edge
                letter_image = image[y - 2:y + h + 2, x - 2:x + w + 2]

                ## Resize the letter to 20*20 for prediction

                letter_image = cv2.resize(letter_image,(20,20))

                ## Turn single image into a 4d array for keras model predict
                letter_image = np.reshape(letter_image, (1,20,20,1))

                ## Predict the letters
                prediction = model.predict(letter_image)

                ## Convert the one-hot-encoded prediction back to a normal letter
                ## return a (1,) ndarray , use [0] to get value
                letter = lb.inverse_transform(prediction)[0]
                predictions.append(letter)

                # print the captcha text

            captcha_text = "".join(predictions)

            return captcha_text
        else:

            return str("Error")


In [0]:
obj = Amazon_Captcha_Recognition()

In [0]:
IMAGE_PATH = "Testing_image/image.jpg"
obj.Captcha_Recognition(IMAGE_PATH)

'ELURNN'